In [187]:
import rioxarray
import pathlib
import numpy

# Replace with your local path

In [188]:
file_path = pathlib.Path("C:\Local\data\VCSN_SamBase_hmax.nc")
nztm2000_epsg=2193
nzgd2000_epsg=4167

# Standard approach to writing CRS information to a NetCDF file where this is missing

In [189]:
file_path = pathlib.Path("C:\Local\data\VCSN_SamBase_hmax.nc")
dem = rioxarray.rioxarray.open_rasterio(file_path)
dem = dem.squeeze("band", drop=True)

C:\Users\pearsonra\Anaconda3\envs\geofabrics\lib\site-packages\rasterio\__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\pearsonra\Anaconda3\envs\geofabrics\lib\site-packages\rioxarray\_io.py:924: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  warnings.warn(str(rio_warning.message), type(rio_warning.message))  # type: ignore


## Write in proper CRS info
Note in the case of a multi-layer netCDF you also need to perform the following for each layer. i.e.
```
dem.layer.rio.write_crs(dem.rio.crs, inplace=True)
dem.layer.rio.write_nodata(numpy.nan, encoded=True, inplace=True)
```

In [190]:
# Write in top level info
dem.rio.set_crs(nztm2000_epsg)
dem.rio.write_crs(dem.rio.crs, inplace=True)
dem.rio.write_transform(inplace=True)

dem.hmax.rio.write_crs(dem.rio.crs, inplace=True)
dem.hmax.rio.write_nodata(numpy.nan, encoded=True, inplace=True);

In [191]:
dem.to_netcdf(file_path.parent / f"{file_path.stem}_crs_{nztm2000_epsg}{file_path.suffix}")
dem.hmax.rio.to_raster(file_path.parent / f"{file_path.stem}_crs_{nztm2000_epsg}.tif")

C:\Users\pearsonra\Anaconda3\envs\geofabrics\lib\site-packages\rasterio\__init__.py:230: NotGeoreferencedWarning: The given matrix is equal to Affine.identity or its flipped counterpart. GDAL may ignore this matrix and save no geotransform without raising an error. This behavior is somewhat driver-specific.
  s = writer(path, mode, driver=driver,


# Convert to Lat and Lon

In [192]:
# Seem to need to set both as otherwise CRS is still the old one even though the reprojection has occured
dem.rio.reproject(nzgd2000_epsg, inplace=True)
dem.rio.write_crs(nzgd2000_epsg, inplace=True);

In [193]:
dem.to_netcdf(file_path.parent / f"{file_path.stem}_crs_{nzgd2000_epsg}{file_path.suffix}")
dem.hmax.rio.to_raster(file_path.parent / f"{file_path.stem}_crs_{nzgd2000_epsg}.tif")

# In the case that the x/y values need to be manually set
Then apply the steps above

In [195]:
file_path = pathlib.Path("C:\Local\data\VCSN_SamBase_hmax.nc")
dem = rioxarray.rioxarray.open_rasterio(file_path)
dem = dem.squeeze("band", drop=True)
dem['x'] = (dem.x + dem.attrs['xmin']) * (dem.attrs['xmax'] - dem.attrs['xmin']) / (float(dem.x.max()) - float(dem.x.min()))
dem['y'] = (dem.y + dem.attrs['ymin']) * (dem.attrs['ymax'] - dem.attrs['ymin']) / (float(dem.y.max()) - float(dem.y.min()))
del dem.attrs['xmin']
del dem.attrs['xmax']
del dem.attrs['ymin']
del dem.attrs['ymax']

C:\Users\pearsonra\Anaconda3\envs\geofabrics\lib\site-packages\rasterio\__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
C:\Users\pearsonra\Anaconda3\envs\geofabrics\lib\site-packages\rioxarray\_io.py:924: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  warnings.warn(str(rio_warning.message), type(rio_warning.message))  # type: ignore
